In [1]:
import os
import json
from dotenv import dotenv_values
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from utils import map_cta_labels, calculate_f1_scores
from utils import save_pickle_file, load_cta_dataset, load_pickle_file, decimal, map_sportstables
import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load env file with API KEY using full path
config = dotenv_values("/full/path/to/file/key.env")
os.environ['OPENAI_API_KEY'] = config["OPENAI_API_KEY"]
OPENAI_API_KEY = config["OPENAI_API_KEY"]

In [3]:
datasets = ["sotabv2", "t2dv2-webtables", "sportstables"]
models = ["gpt-3.5-turbo-0301", "gpt-4-0613"]

## Manual definitions

In [ ]:
for dataset in datasets:
    print(dataset)
    # Load dataset
    examples, labels, test_table_type_labels, train_examples, train_example_labels, train_table_type_labels, labels_to_text, text_to_label, labels_joined, train, test = load_cta_dataset(dataset,"")
    all_labels = [labels_to_text[l] for l in labels_to_text]

    f = open(f'../data/{dataset}-definitions.txt')
    definitions = json.load(f)
    all_labels = [labels_to_text[defn] for defn in definitions]
    definitions = [definitions[defn] for defn in definitions]
    test_embeddings = load_pickle_file(f'embeddings/test_embeddings_{dataset}.pkl')

    # Pick the necessary deinitions for each example
    knowledge_embeddings = load_pickle_file(f"embeddings/{dataset}-definitions-embeddings.pkl")
    examples_demonstrations = []
    for i, example in tqdm.tqdm(enumerate(examples), total=len(examples)):
        cos = cosine_similarity([test_embeddings[i]], knowledge_embeddings)
        cos_dict = {}
        for j, c in enumerate(cos[0]):
            cos_dict[j] = c
        sorted_cos_dict = {k: v for k, v in sorted(cos_dict.items(), key=lambda item: item[1])}
        examples_demonstrations.append(list(sorted_cos_dict.keys())[-10:])
        
    for model_name in ["gpt-3.5-turbo-0301","gpt-4-0613"]:
        print(model_name)
        chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model=model_name)
        preds = []

        for j, example in tqdm.tqdm(enumerate(examples), total=len(examples)):

            messages = []

            knowledge_string = f""
            for index in examples_demonstrations[j]:
                knowledge_string += f"{all_labels[index]}: {definitions[index]}\n"
            knowledge_string = knowledge_string.strip()

            messages.append(SystemMessage(content=knowledge_string))

            #Task and instructions
            messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate the columns of a given table with only one of the following labels that are separated with comma: {labels_joined}."))
            messages.append(SystemMessage(content="Your instructions are: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values in detail. 3. For each column, select a label that best represents the meaning of all cells in the column. 4. Answer with the selected label for each column using the format Column1: label. 5. Answer only with labels from the provided label set!"))#
            messages.append(HumanMessage(content=f"Classify these table columns: {example}"))

            res = chat(messages)
            preds.append(res.content)

        save_pickle_file(f"predictions/{dataset}/{model_name}/manual-definitions-table-0-shot.pkl", preds)

## LLM definitions experiments

In [40]:
model_name = 'gpt-3.5-turbo-0301'
# model_name = 'gpt-4-0613'
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model=model_name)

In [41]:
a_prompts = ["A1", "A2", "A3", "A4", "A5", "A6"]
tb_prompts = ["TB1", "TB2", "TB3", "TB4", "TB5", "TB6", "TB7"]
inst_prompts = ["I1", "I2"]
syst_prompts = ["S1", "S2", "S3", "S4", "S5"]

In [ ]:
for dataset in datasets[:2]:
    print(dataset)
    # Load dataset
    examples, labels, test_table_type_labels, train_examples, train_example_labels, train_table_type_labels, labels_to_text, text_to_label, labels_joined, train, test = load_cta_dataset(dataset,"")
    all_labels = [labels_to_text[l] for l in labels_to_text]

    # Load embeddings
    test_embeddings = load_pickle_file(f"embeddings/test_embeddings_{dataset}.pkl")
    
    # Run Test A prompts
    for syst in syst_prompts:
        for g in a_prompts:
            if f"{syst}_{g}-table-0-shot.pkl" not in os.listdir(f"predictions/{dataset}/{model_name}/"):
                print(f"Loading knowledge {syst}_{g}")
                definitions = load_pickle_file(f"knowledge/{model_name}/{dataset}/{syst}_{g}_prompt_knowledge.pkl")
                knowledge_embeddings = load_pickle_file(f"embeddings/{model_name}/{syst}_{g}_knowledge_embeddings_{dataset}.pkl")
                
                examples_demonstrations = []
                for i, example in tqdm.tqdm(enumerate(examples), total=len(examples)):
                    cos = cosine_similarity([test_embeddings[i]], knowledge_embeddings)
                    cos_dict = {}
                    for j, c in enumerate(cos[0]):
                        cos_dict[j] = c
                    sorted_cos_dict = {k: v for k, v in sorted(cos_dict.items(), key=lambda item: item[1])}
                    examples_demonstrations.append(list(sorted_cos_dict.keys())[-10:])
                try:
                    preds = []

                    for j, example in tqdm.tqdm(enumerate(examples), total=len(examples)):

                        messages = []

                        knowledge_string = f""
                        for index in examples_demonstrations[j]:
                            knowledge_string += f"{all_labels[index]}: {definitions[index]}\n"
                        knowledge_string = knowledge_string.strip()

                        messages.append(SystemMessage(content=knowledge_string))
                        #Task and instructions
                        messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate the columns of a given table with only one of the following labels that are separated with comma: {labels_joined}."))
                        messages.append(SystemMessage(content="Your instructions are: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values and definitions given to you in detail. 3. For each column, select a label that best represents the meaning of all cells in the column. 4. Answer with the selected label for each column using the format Column1: label. 5. Answer only with labels from the provided label set!"))
                        messages.append(HumanMessage(content=f"Classify these table columns: {example}"))

                        res = chat(messages)
                        preds.append(res.content)

                    save_pickle_file(f"predictions/{dataset}/{model_name}/{syst}_{g}-table-0-shot.pkl", preds)
                except Exception:
                    print(f"Error in {syst}_{g}_{dataset}.pkl")
                    continue
                    
    
    # Run B prompts             
    for tb in tb_prompts:
        for syst in syst_prompts[:-1]: # skip last system for prompt b
            for inst in inst_prompts:
                if f"{syst}_{inst}_{tb}-table-0-shot.pkl" not in os.listdir(f"predictions/{dataset}/{model_name}/"):
                    print(f"Loading knowledge {tb}_{syst}_{inst}")
                    definitions = load_pickle_file(f"knowledge/{model_name}/{dataset}/{syst}_{inst}_{tb}_prompt_knowledge.pkl")
                    knowledge_embeddings = load_pickle_file(f"embeddings/{model_name}/{syst}_{inst}_{tb}_knowledge_embeddings_{dataset}.pkl")

                    examples_definitions = []
                    for i, example in tqdm.tqdm(enumerate(examples), total=len(examples)):
                        cos = cosine_similarity([test_embeddings[i]], knowledge_embeddings)
                        cos_dict = {}
                        for j, c in enumerate(cos[0]):
                            cos_dict[j] = c
                        sorted_cos_dict = {k: v for k, v in sorted(cos_dict.items(), key=lambda item: item[1])}
                        examples_definitions.append(list(sorted_cos_dict.keys())[-10:])

                    try:
                        preds = []

                        for j, example in tqdm.tqdm(enumerate(examples), total=len(examples)):

                            messages = []

                            knowledge_string = f""
                            for index in examples_definitions[j]:
                                knowledge_string += f"{all_labels[index]}: {definitions[index]}\n"
                            knowledge_string = knowledge_string.strip()

                            messages.append(SystemMessage(content=knowledge_string))

                            #Task and instructions
                            messages.append(SystemMessage(content=f"You are a world-class data engineer and your task is to annotate the columns of a given table with only one of the following labels that are separated with comma: {labels_joined}."))
                            messages.append(SystemMessage(content="Your instructions are: 1. Look at the input given to you and make a table out of it. 2. Look at the cell values and definitions given to you in detail. 3. For each column, select a label that best represents the meaning of all cells in the column. 4. Answer with the selected label for each column using the format Column1: label. 5. Answer only with labels from the provided label set!"))
                            messages.append(HumanMessage(content=f"Classify these table columns: {example}"))

                            res = chat(messages)
                            preds.append(res.content)

                        save_pickle_file(f"predictions/{dataset}/{model_name}/{syst}_{inst}_{tb}-table-0-shot.pkl", preds)
                    except Exception:
                        print(f"Error in: {syst}_{inst}_{tb}_{dataset}.pkl")
                        continue
                        

## Evaluation

In [ ]:
for dataset in datasets[:2]:
    print(dataset)
    print(f"\tPrecision)\tRecall\tMacro-F1\tMicro-F1\tOOV")
    # Load dataset
    examples, labels, test_table_type_labels, train_examples, train_example_labels, train_table_type_labels, labels_to_text, text_to_label, labels_joined, train, test = load_cta_dataset(dataset,"")
    if dataset == "sportstables":
        labels = [l for l in labels if l != ""]
    
    for tb in tb_prompts:
        for syst in syst_prompts[:-1]: # skip last system for prompt b
            for inst in inst_prompts:
                try:
                    preds = load_pickle_file(f"predictions/{dataset}/{model_name}/{syst}_{inst}_{tb}-table-0-shot.pkl")
                    types = list(set(labels))

                    if dataset == "sportstables":
                        predictions, num = map_sportstables(preds, types, test)
                    else:
                        predictions, num = map_cta_labels(preds, test, text_to_label)
                    
                    types = types+["-"] if '-' in predictions else types
                    evaluation, per_class_eval = calculate_f1_scores(labels, predictions, len(types), types)

                    print(f"{syst}_{inst}_{tb}\t{decimal(evaluation['Precision'])}\t{decimal(evaluation['Recall'])}\t{decimal(evaluation['Macro-F1'])}\t{decimal(evaluation['Micro-F1'])}\t{num}")
                except Exception:
                    print(f"Error in: {syst}_{inst}_{tb}-table-0-shot.pkl")
                    
    for syst in syst_prompts:
        for g in a_prompts:
            try:
                preds = load_pickle_file(f"predictions/{dataset}/{model_name}/{syst}_{g}-table-0-shot.pkl")
                types = list(set(labels))

                if dataset == "sportstables":
                    predictions, num = map_sportstables(preds, types, test)
                else:
                    predictions, num = map_cta_labels(preds, test, text_to_label)

                types = types+["-"] if '-' in predictions else types
                evaluation, per_class_eval = calculate_f1_scores(labels, predictions, len(types), types)

                print(f"{syst}_{g}\t{decimal(evaluation['Precision'])}\t{decimal(evaluation['Recall'])}\t{decimal(evaluation['Macro-F1'])}\t{decimal(evaluation['Micro-F1'])}\t{num}")
            except Exception:
                print(f"Error in: {syst}_{g}-table-0-shot.pkl")